In [1]:
from beta_VAE import beta_VAE
beta_VAE = beta_VAE()

Load_image_dataset?[yes/no] yes


100%|██████████| 20608/20608 [00:15<00:00, 1349.57it/s]


loading completed.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use keras.layers.batch_normalization instead.
Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use tf.cast instead.


In [2]:
beta_VAE.fit(epoch=50)
beta_VAE.Save()

  0%|          | 0/50 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [8]:
beta_VAE.Save()

In [9]:
beta_VAE.load()

INFO:tensorflow:Restoring parameters from /home/baxter/Documents/beta-vae/checkpoints/model.ckpt
